
# SmartSales – Análise Exploratória (EDA) e KPIs

Este notebook roda **no Colab** ou **local (VS Code/Jupyter)**. A primeira célula:
1. **Instala dependências** se faltarem.
2. Faz **autenticação híbrida** (Colab = sua conta Google; Local = Service Account via `GDRIVE_SA_JSON`).
3. Se encontrar `data/samples/SmartSales_Dados.csv`, roda em **modo offline** com **dimensões sintéticas** (Produtos, Clientes, Vendedores) geradas de forma determinística a partir dos IDs.


In [1]:

# 🚀 Bootstrap: deps + auth + utils
import os, sys, subprocess, importlib, hashlib, math, random
import pandas as pd

REQUIRED = ["pandas","gspread","google-auth","google-auth-oauthlib","matplotlib"]
def ensure(pkgs):
    miss=[]
    for p in pkgs:
        try: importlib.import_module(p.replace("-","_"))
        except ModuleNotFoundError: miss.append(p)
    if miss:
        print("Instalando dependencias:", ", ".join(miss))
        subprocess.check_call([sys.executable,"-m","pip","install",*miss])
    else:
        print("Dependencias OK.")
ensure(REQUIRED)

import matplotlib.pyplot as plt

def br_money(x):
    try: return "R$ " + f"{float(x):,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
    except Exception: return str(x)

def padronizar_colunas(df, candidatos, destino):
    for c in candidatos:
        if c in df.columns and destino not in df.columns:
            df.rename(columns={c: destino}, inplace=True); break

def aplicar_canonicos(vendas=None, produtos=None, clientes=None, vendedores=None):
    if produtos is not None:
        padronizar_colunas(produtos, ["produto_id","product_id","id_produto","id"], "_product_id")
        padronizar_colunas(produtos, ["produto","product","nome_produto","descricao","name"], "_product_nm")
        padronizar_colunas(produtos, ["preco_venda","preco","price","valor"], "price")
        padronizar_colunas(produtos, ["custo","cost"], "cost")
    if vendas is not None:
        padronizar_colunas(vendas, ["id_venda","order_id","pedido"], "order_id")
        padronizar_colunas(vendas, ["id_produto","produto_id","product_id","id_produto"], "_product_id")
        padronizar_colunas(vendas, ["id_cliente","cliente_id","client_id"], "_client_id")
        padronizar_colunas(vendas, ["id_vendedor","vendedor_id","seller_id"], "_seller_id")
        padronizar_colunas(vendas, ["quantidade","qtd","qty"], "qty")
        padronizar_colunas(vendas, ["preco","price","valor"], "price")
        padronizar_colunas(vendas, ["data","date","data_venda","dt_venda"], "date")
    if clientes is not None:
        padronizar_colunas(clientes, ["cliente_id","client_id","id_cliente"], "_client_id")
        padronizar_colunas(clientes, ["cliente","nome_cliente","client","name"], "_client_nm")
        padronizar_colunas(clientes, ["cidade","city","municipio"], "_city")
    if vendedores is not None:
        padronizar_colunas(vendedores, ["vendedor_id","seller_id","id_vendedor"], "_seller_id")
        padronizar_colunas(vendedores, ["vendedor","seller","nome_vendedor","name"], "_seller_nm")
    return vendas, produtos, clientes, vendedores

def validar_dados(vendas, produtos):
    issues = {}
    if "qty" in vendas.columns: issues["qtd_nao_positiva"] = int((vendas["qty"] <= 0).sum())
    if "price" in vendas.columns: issues["preco_venda_negativo"] = int((vendas["price"] < 0).sum())
    if "date" in vendas.columns:
        try: pd.to_datetime(vendas["date"], errors="raise"); bad=0
        except Exception:
            parsed = pd.to_datetime(vendas["date"], errors="coerce"); bad=int(parsed.isna().sum())
        issues["datas_invalidas"] = bad
    for col in ["cost","price"]:
        if produtos is not None and col in (produtos.columns if hasattr(produtos,'columns') else []):
            issues[f"produtos_sem_{col}"] = int(produtos[col].isna().sum())
    print("Checklist de validacao:", issues); return issues

print("Bootstrap OK.")


ModuleNotFoundError: No module named 'pandas'

## Carregamento dos dados

In [ ]:

csv_path = "data/samples/SmartSales_Dados.csv"
use_offline = os.path.exists(csv_path)

if use_offline:
    print("Modo offline: carregando", csv_path)
    vendas = pd.read_csv(csv_path)
    # dimensoes sinteticas deterministicas
    rng_cidades = ["Porto Alegre","Caxias do Sul","Pelotas","Santa Maria","Canoas","Novo Hamburgo"]
    def dhash(s):  # hash deterministico 0..1
        h = hashlib.md5(str(s).encode()).hexdigest()
        return int(h[:8],16) / 0xFFFFFFFF

    # produtos
    prod_ids = sorted(vendas["id_produto"].dropna().unique())
    produtos = pd.DataFrame({
        "_product_id": prod_ids,
        "_product_nm": [f"Produto {pid}" for pid in prod_ids],
    })
    # preco deterministico 20..300
    produtos["price"] = [round(20 + dhash(pid)*280, 2) for pid in produtos["_product_id"]]
    # custo = 60%..85% do preco
    produtos["cost"]  = [round(p * (0.6 + dhash(pid)*0.25), 2) for p,pid in zip(produtos["price"], produtos["_product_id"])]

    # clientes
    cli_ids = sorted(vendas["id_cliente"].dropna().unique())
    clientes = pd.DataFrame({
        "_client_id": cli_ids,
        "_client_nm": [f"Cliente {cid}" for cid in cli_ids],
        "_city": [rng_cidades[ int(math.floor(dhash(cid)*len(rng_cidades))) ] for cid in cli_ids]
    })

    # vendedores
    ven_ids = sorted(vendas["id_vendedor"].dropna().unique())
    vendedores = pd.DataFrame({
        "_seller_id": ven_ids,
        "_seller_nm": [f"Vendedor {vid}" for vid in ven_ids]
    })

    # padroniza vendas
    vendas, produtos, clientes, vendedores = aplicar_canonicos(vendas, produtos, clientes, vendedores)
    # cria price se nao existir em vendas (usa da dimensao apos merge)
    # formata date
    if "date" in vendas.columns:
        vendas["date"] = pd.to_datetime(vendas["date"], errors="coerce")
    else:
        pass
    print("Tabelas carregadas: vendas, produtos, clientes, vendedores (sinteticos).")
else:
    print("Modo Google Sheets: autentique (Colab: automatico; Local: Service Account).")
    # === Exemplo de como abrir uma planilha ===
    # from google.colab import auth  # no Colab
    # sh = gc.open_by_url("URL_DA_PLANILHA")
    # vendas = pd.DataFrame(sh.worksheet("Vendas").get_all_records())
    # produtos = pd.DataFrame(sh.worksheet("Produtos").get_all_records())
    # clientes = pd.DataFrame(sh.worksheet("Clientes").get_all_records())
    # vendedores = pd.DataFrame(sh.worksheet("Vendedores").get_all_records())
    # vendas, produtos, clientes, vendedores = aplicar_canonicos(vendas, produtos, clientes, vendedores)


## Enriquecimento e checagens

In [ ]:

# Merge de vendas com produtos para trazer price/cost se necessario
if use_offline:
    vendas_full = vendas.merge(produtos[["_product_id","price","cost"]], on="_product_id", how="left", suffixes=("","_prod"))
    # qty obrigatoria
    if "qty" not in vendas_full.columns and "quantidade" in vendas.columns:
        vendas_full.rename(columns={"quantidade":"qty"}, inplace=True)
else:
    vendas_full = vendas.copy()

issues = validar_dados(vendas_full, produtos)
vendas_full["receita"] = vendas_full["qty"] * vendas_full["price"]
vendas_full["custo_total"] = vendas_full["qty"] * vendas_full["cost"]
vendas_full["lucro"] = vendas_full["receita"] - vendas_full["custo_total"]


## KPIs principais

In [ ]:

kpis = {}
kpis["receita_total"] = vendas_full["receita"].sum()
kpis["lucro_total"] = vendas_full["lucro"].sum()
kpis["qtd_pedidos"] = vendas_full["order_id"].nunique() if "order_id" in vendas_full.columns else len(vendas_full)
kpis["qtd_itens"] = int(vendas_full["qty"].sum())

print("Receita:", br_money(kpis["receita_total"]))
print("Lucro:", br_money(kpis["lucro_total"]))
print("Pedidos:", kpis["qtd_pedidos"])
print("Itens vendidos:", kpis["qtd_itens"])

top_prod = (vendas_full.groupby("_product_id", as_index=False)["receita"].sum()
            .merge(produtos[["_product_id","_product_nm"]], on="_product_id", how="left")
            .sort_values("receita", ascending=False).head(10))
top_prod.head(10)


## Compras por cidade

In [ ]:

if "_client_id" in vendas_full.columns:
    tmp = vendas_full.merge(clientes[["_client_id","_city"]], on="_client_id", how="left")
    if "order_id" in tmp.columns:
        compras_por_cidade = (tmp.dropna(subset=["_city"])
                                .groupby("_city")["order_id"].nunique()
                                .reset_index(name="qtd_compras")
                                .sort_values("qtd_compras", ascending=False))
    else:
        compras_por_cidade = (tmp.dropna(subset=["_city"]).groupby("_city", as_index=False).size()
                                .rename(columns={"size":"qtd_compras"})
                                .sort_values("qtd_compras", ascending=False))
    compras_por_cidade.head(10)
else:
    print("Sem _client_id para calcular compras por cidade.")


## Gráfico – Top produtos por receita

In [ ]:

plt.figure(figsize=(8,4))
plot_df = top_prod.head(8)
plt.bar(plot_df["_product_nm"].astype(str), plot_df["receita"])
plt.title("Top 8 produtos por receita")
plt.xticks(rotation=45, ha="right")
plt.ylabel("Receita (R$)")
plt.tight_layout()
plt.show()
